In [2]:
import pandas as pd
import glob

In [3]:
import spacy
from spacy import displacy
from spacy.lang.en import English
nlp = spacy.load("en_core_web_sm")
# from spacy.gold import GoldParse
from spacy.pipeline import EntityRecognizer

In [ ]:
# Get path for all tweet files to concatenate them
path = r'./elon_musk_tweets'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    # print(filename)
    frame = pd.read_csv(filename, index_col=None, encoding="utf-8")
    li.append(frame)

# print(li)
df = pd.concat(li, axis=0, ignore_index=True)
df.shape
# df[35000:35101]
# df.head(30)
# df.date

In [ ]:
df_elon = df[["date","tweet"]]
# df = df.drop(['id','conversation_id','created_at','timezone','place','language','near','geo','source','user_rt_id','user_rt', 'retweet_id','reply_to','translate','trans_dest','cashtags','user_id', 'user_id_str','day','video','thumbnail','quote_url','search','trans_src','link','urls', 'photos'], 1)
# df.date[30550]
# df[:10]
df_elon.tail(10)

In [ ]:
# Saving tweets to a list
dates = df["date"].to_list()
print(dates[-1])
tweets = df["tweet"].to_list()
len(tweets)

In [ ]:
# # function to find entities with spacy and see their labels and explanation
# def show_ents(doc):
#     doc = nlp(doc)
#     if doc.ents:
#         for ent in doc.ents:
#             print(ent.text+" - "+str(ent.start_char) +" - "+ str(ent.end_char) +" - "+ent.label_+ " - "+str(spacy.explain(ent.label_)))
#     else:
#         print("No named entities found.")

In [ ]:
# # function check
# show_ents(tweets[253])

In [7]:
# Function to collect a dictionary with ORG entities mentioned, tweets and dates

def mentioned_orgs(tweets):
    """Function takes in a list of tweets and returns a dictionary with organisation's name as a key and a list of tweets and dates of posting as a value"""
    elon_companies = {}
    count = 0
    for doc in tweets:
        doc = nlp(doc) # text processing
        if doc.ents:
        # searching for named entities that stand for ORG
            for ent in doc.ents:
                if ent.label_=="ORG":
                # print(ent, type(ent))
                    ent = str(ent)
                    if ent in elon_companies:
                        elon_companies[ent].append(list())
                        elon_companies[ent][-1].append(dates[count])
                        elon_companies[ent][-1].append(tweets[count]) #adding dates to the value
                    else:
                        elon_companies[ent] = list()
                        elon_companies[ent].append(list())
                        elon_companies[ent][0].append(dates[count])
                        elon_companies[ent][0].append(tweets[count])
        count += 1

    return elon_companies

In [8]:
companies = mentioned_orgs(tweets)

In [ ]:
# dctionary to count times each company was mentioned
elon_top_companies = {}
for k in companies:
    elon_top_companies[k] = len(companies[k])
    # print(k, len(elon_companies[k]))
    
print(elon_top_companies)
# print(len(elon_companies["Tesla"]), elon_companies["Tesla"])

In [ ]:
# sorting dictionary in descending order
elon_top_companies = sorted(elon_top_companies.items(), key=lambda x: x[1], reverse=True)
print("Companies Elon Musk most talks about:\n")
count = 1
for k,v in elon_top_companies[:21]:
    print("{}. {} ({} tweets)".format(count, k, v))
    count += 1

In [22]:
# comp_products = {
    # "Model S": "Tesla"
# }

comp_products = {
    "Zip2": "Zip2",
    "Falcon": "SpaceX",
    "Falcon Heavy": "SpaceX",
    "Falcon 9": "SpaceX",
    "Grasshopper": "SpaceX", 
    "Starship": "SpaceX", 
    "Merlin": "SpaceX", 
    "Kestrel": "SpaceX", 
    "Raptor": "SpaceX", 
    "Methox thruster": "SpaceX", 
    "Draco": "SpaceX", 
    "SuperDraco": "SpaceX", 
    "Dragon": "SpaceX", 
    "Cargo Dragon": "SpaceX", 
    "Starlink": "SpaceX", 
    "Hyperloop": "SpaceX", 
    "vactrain": "SpaceX",
    "Model S": "Tesla",
    "Roadster": "Tesla",
    "Model X": "Tesla", 
    "Model 3": "Tesla", 
    "Model Y": "Tesla", 
    "Semi": "Tesla", 
    "Cybertruck": "Tesla", 
    "Powerwall": "Tesla", 
    "Solar Roof": "Tesla", 
    "Solar Inverter": "Tesla", 
    "Powerpack": "Tesla", 
    "Megapack": "Tesla", 
    "Supercharger": "Tesla", 
    "Destination charger": "Tesla", 
    "Autopilot": "Tesla", 
    "Full self-driving": "Tesla", 
    "FSD": "Tesla",
    "Application-Specific Integrated Circuit": "Neuralink",
    "ASIC": "Neuralink", 
    "brain-computer interface": "Neuralink",
    "Hawthorne tunnel": "The Boring Company", 
    "Los Angeles tunnel": "The Boring Company", 
    "Loop": "The Boring Company", 
    "Loop System": "The Boring Company",
    "Gym": "OpenAI", 
    "RoboSumo": "OpenAI", 
    "Debate Game": "OpenAI", 
    "Dactyl": "OpenAI", 
    "GPT": "OpenAI", 
    "generative pre-training": "OpenAI", 
    "GPT-2": "OpenAI", 
    "GPT-3": "OpenAI", 
    "MuseNet": "OpenAI", 
    "API": "OpenAI", 
    "DALL-E": "OpenAI", 
    "CLIP": "OpenAI", 
    "Microscope": "OpenAI", 
    "Codex": "OpenAI", 
    "Five": "OpenAI", 
    "GYM Retro": "OpenAI"
}

# print(comp_products["Model S"])

In [23]:
# Cleaning data processed with Spacy checking the entities against the names of companies' products.
new_d = {}
for k in companies:
    if k in comp_products:
        # print(k)
        # print(comp_products[k])
        # print(companies[k])
        if comp_products[k] not in new_d:
            new_d[comp_products[k]] = companies[k]
        else:
            new_d[comp_products[k]].extend(companies[k])
    else:
        if k in new_d:
            # print(k)
            new_d[k].extend(companies[k])
        else:
            # print(k)
            new_d[k] = companies[k]

# print(len(new_d["Tesla"]))

In [ ]:
# dictionary to count times each company was mentioned
new_d_count = {}
for k in new_d:
    new_d_count[k] = len(new_d[k])
    # print(k, len(elon_companies[k]))
    
# print(new_d_count)

new_d_sorted = sorted(new_d_count.items(), key=lambda x: x[1], reverse=True)
print("Companies Elon Musk most talks about:\n")
count = 1
for k,v in new_d_sorted[:31]:
    print("{}. {} ({} tweets)".format(count, k, v))
    count += 1

In [ ]:
# count = 0
# for k in companies:
#     for c in comp_products:
#         for item in comp_products[c]:
#             if k == item:
#                 count += 1
#                 print(k, "vs.", item, "\n")
# print(count)

In [47]:
def company_tweets_to_list(companies_dict, company_name):
    """Function takes in a dictionary with tweets about companies and a string with the name of a company under consideration"""
    comp_tweets = []

    idx = 0
    while idx in range(len(companies_dict[company_name])):
        comp_tweets.append(companies_dict[company_name][idx][1])
        idx += 1

    return comp_tweets
    # for key in new_d:
    #     idx = 0
    #     while idx in range(len(new_d[key])):
    #         comp_tweets.append(new_d[key][idx][1])
    #         idx += 1
  

In [ ]:
tesla_tweets = company_tweets_to_list(new_d, company_name="Tesla")
# tesla_tweets[:2]

In [58]:
spacex_tweets = company_tweets_to_list(new_d, company_name="SpaceX")

In [62]:
nasa_tweets = company_tweets_to_list(new_d, company_name="@NASA")

In [66]:
tbc_tweets = company_tweets_to_list(new_d, company_name="The Boring Company")

In [68]:
google_tweets = company_tweets_to_list(new_d, company_name="Google")

In [72]:
apple_tweets = company_tweets_to_list(new_d, company_name="Apple")

In [37]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [54]:
def sentiment_analyzer(lst_texts):
    """Function takes in a list of texts and returns a dictionary with keys as numbers and text and sentiment scores with explanation for a value"""
    data = {}
    analyzer = SentimentIntensityAnalyzer()
    count = 0
    for text in lst_texts:
        vs = analyzer.polarity_scores(text)
        data[count] = list()
        data[count].extend([text, vs])
        data[count].append(data[count][1]['compound'])
        if data[count][1]['compound'] > 0:
            data[count].append("POSITIVE")
        elif data[count][1]['compound'] == 0:
            data[count].append("NEUTRAL")
        else:
            data[count].append("NEGATIVE")
        count += 1
    # print(data[3000])
    return data

In [55]:
tesla = sentiment_analyzer(tesla_tweets)

In [ ]:
tesla_df = pd.DataFrame.from_dict(tesla, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
tesla_df

In [ ]:
tesla_df.Sentiment_type.value_counts().plot(kind="bar", title="Tesla Sentiment Analysis", color="#55FF33")

In [ ]:
spacex = sentiment_analyzer(spacex_tweets)
spacex_df = pd.DataFrame.from_dict(spacex, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
spacex_df
spacex_df.Sentiment_type.value_counts().plot(kind='bar',title="SpaceX sentiment analysis", color="#3396FF")

In [ ]:
nasa = sentiment_analyzer(nasa_tweets)
nasa_df = pd.DataFrame.from_dict(nasa, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
nasa_df
nasa_df.Sentiment_type.value_counts().plot(kind='bar',title="NASA Sentiment Analysis")

In [ ]:
tbc = sentiment_analyzer(tbc_tweets)
tbc_df = pd.DataFrame.from_dict(tbc, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
tbc_df
tbc_df.Sentiment_type.value_counts().plot(kind='bar',title="The Boring Company Sentiment Analysis")

In [ ]:
google = sentiment_analyzer(google_tweets)
google_df = pd.DataFrame.from_dict(google, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
google_df
google_df.Sentiment_type.value_counts().plot(kind='bar',title="Google Sentiment Analysis")

In [ ]:
apple = sentiment_analyzer(apple_tweets)
apple_df = pd.DataFrame.from_dict(apple, orient="index",columns=["Tweet","Scores", "Compound", "Sentiment_type"])
apple_df
apple_df.Sentiment_type.value_counts().plot(kind='bar',title="Apple Sentiment Analysis")